# Reach Curve Modeling Based On Timespends Data
 
This notebook explores and models reach curve.

## Objectives:
1. Load and explore data
2. Exploratory data analysis
3. Research
3. Reach curve modeling
4. Evaluation method
5. Reach Optimization
6. Recommendations and Insights

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn_extra.cluster import KMedoids

import warnings
warnings.filterwarnings('ignore')

## 1. Load and explore data

In [ ]:
data = pd.read_csv("../data/timespends.csv", index_col=0)

TOTAL_NUM_SAMPLES = len(data)
print(f"No. of samples: {TOTAL_NUM_SAMPLES}")
print(f"Sample:\n {data.head()}")


plt.plot(sorted(data['timespends']))
plt.title("Timespends (sorted)")
plt.show()

### Basic observations

So, the entire population comprises 10,333 samples, for which the average time spent on a particular medium was measured.

The above plot reveals about 20% of sampled population doesn't use the medium at all. 
This means there's simply no way to reach them through this medium, and it will also set an upper limit for the reach curve (around 80%).

On the other hand, about 15% of the population spends the most time on this medium compared to all other groups. 
This group will likely contain the highest number of individuals who will see an advertisement multiple times. 
Because of it - the reach curve will flatten – despite an increase in the number of impressions.

## 2. Exploratory Data Analysis

In [ ]:
data['timespends'].value_counts().head(15)

In [ ]:
print("Zero time users are {:.2f} % of all users.".format(np.sum(data['timespends'] == 0) / len(data) * 100))

In [ ]:
N_CLUSTERS = 7
X = data['timespends'].to_numpy().reshape(-1, 1)
kmedoids = KMedoids(n_clusters=N_CLUSTERS, random_state=0, method='pam').fit(X)

In [ ]:
clustered_data = data.copy()
clustered_data['cluster'] = kmedoids.predict(X)
clustered_data['cluster-timespend'] = clustered_data.apply(lambda row: kmedoids.cluster_centers_[int(row['cluster'])][0], axis=1)
clustered_data.head()

In [ ]:
plt.plot(sorted(clustered_data['cluster-timespend']))
plt.title("Timespends (sorted) - clustered")

In [ ]:
df = pd.DataFrame(clustered_data['cluster-timespend'].value_counts())
df.reset_index(inplace=True)
df.sort_values(by='cluster-timespend', inplace=True, ascending=False)
df['perc_of_total'] = df['count'] / TOTAL_NUM_SAMPLES * 100
df['perc_of_total_cum'] = np.cumsum(df['perc_of_total'].to_numpy())
df

### Observations
1. We have 7 main user groups defined.
2. The maximum value the reach curve will achieve is ```79.45%```
3. The minimum value is 0 (it would be interesting to estimate this worst-case scenario somehow — i.e., what would be the chances that absolutely no one gets any impression even once).
4. The reach curve is a non-decreasing curve because an impression, once received, cannot be unreceived (or taken back).

## 3. Research

### Articles - a brief literature overview
1. [Reach Measurement, Optimization and Frequency Capping In Targeted Online Advertising Under k-Anonymity](https://arxiv.org/pdf/2501.04882v1)
2. [Estimating reach curves from one data point](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/43218.pdf)
3. [Privacy-centric Cross-publisher Reach and Frequency Estimation Via Vector of Counts](https://storage.googleapis.com/gweb-research2023-media/pubtools/6039.pdf#cite.kreuter2020privacy)
4. [Privacy-Preserving Secure Cardinality and Frequency Estimation](https://storage.googleapis.com/gweb-research2023-media/pubtools/5611.pdf)
5. [Virtual People: Actionable Reach Modeling](https://research.google/pubs/virtual-people-actionable-reach-modeling/)
6. [Measuring Cross-Device Online Audiences](https://research.google/pubs/measuring-cross-device-online-audiences/)
7. [Scalable Multi-objective Optimization in Programmatic Advertising via Feedback Control](https://www.researchgate.net/publication/356879928_Scalable_Multi-objective_Optimization_in_Programmatic_Advertising_via_Feedback_Control)

These are articles I found while searching for topics related to reach curve modeling. While there's only one curve, it's a surprisingly broad subject with fascinating modeling.

## 4. Simulation of reach curve
To better understand the problem, it's worth considering running a simulation.

The idea is to treat impressions as something requested by a given user who uses a particular medium - this user is sampled from the entire set of users.

It's important to note that the sampling will not include users who do not use the given medium.

### Mathematical Framework

Let's define our mathematical framework:

- **User space**: $\Omega = \{0, 1, ..., 10332\}$ - all users
- **Random variable for time spent**: $X: \Omega \to \mathbb{R}$, where $X(\omega_i) = $ average time spent by user $i$
- **Random variable for impression probability**: $Y: \Omega \to \mathbb{R}$, where $Y(\omega_i) = p_i$ - i.e. probability that user $i$ requested impression
- **Conditional probability**: $P(Y = i | X > 0) = \frac{P(X > 0 | Y = i) \cdot P(Y = i)}{P(X > 0)}$ - probability that user i requested impression considering that nonzero spending time.

## 1. Mathematical Framework

### State Space and Random Variables
- $\Omega = \{0, 1, ..., N - 1\}$ - user population, where $N = 10333$
- $X: \Omega -> \mathbb{R}$ - random variable $X$ to annotate timespend of user $i$
- $\mathcal{A} = \{\omega_i \in \Omega : X(\omega_i) > 0\}$ - active users who can see ads
- $S_n \subseteq \Omega$ - set of users reached after $n$ impressions
- $R(n) = |S_n|$ - reach function (or $|S_n| / |\Omega| * 100$%)

### Impression Allocation Process
Each impression $n$ selects user $U_n$ with probability:
$$P(U_n = i) = \begin{cases}
\frac{X_i}{\sum_{j \in \mathcal{A}} X_j} & \text{if } i \in \mathcal{A} \\
0 & \text{otherwise}
\end{cases}$$

### Reach Evolution
- $S_0 = \emptyset$
- $S_{n+1} = S_n \cup \{U_n\}$
- $R(n) = |S_n|$ is non-decreasing with $R(n) \leq |\mathcal{A}|$

### Simulation

In [ ]:
class StochasticReachSimulator:
    """
    Simulates reach curve using stochastic process based on user timespends.

    Note: Probabilities for active users are 
    directly related to their time spent on the medium.
    """
    
    def __init__(self, timespends):
        self.timespends = timespends
        self.total_users = len(timespends)
        
        self.active_mask = timespends > 0
        self.active_indices = np.where(self.active_mask)[0]
        self.active_timespends = timespends[self.active_mask]
        
        # TODO: it's good idea to think out about some startegy
        self.probabilities = self.active_timespends / np.sum(self.active_timespends)
        self.max_reach = len(self.active_indices)
        
    def simulate_single_run(self, n_impressions: int) -> tuple[np.array, set[int]]:
        """
        Simulate a single run of the reach process.
        
        Returns:
            reach_curve: array of reach values at each impression
            reached_users: set of users reached

        """
        reached_users = set()
        reach_curve = np.zeros(n_impressions + 1)
        
        for i in range(n_impressions):
            selected_user_idx = np.random.choice(
                self.active_indices, 
                p=self.probabilities
            )
            
            reached_users.add(selected_user_idx)
            reach_curve[i + 1] = len(reached_users)
            
        return reach_curve, reached_users
    
    def simulate_multiple_runs(self, n_impressions: int, n_runs: int) -> np.array:
        """
        Simulate multiple runs to get distribution of reach curves.
        """
        reach_curves = np.zeros((n_runs, n_impressions + 1))
        
        for run in tqdm(range(n_runs), desc="Simulating runs"):
            reach_curve, _ = self.simulate_single_run(n_impressions)
            reach_curves[run] = reach_curve
            
        return reach_curves
    
    def compute_expected_reach(self, n_impressions: int) -> np.array:
        """
        Compute theoretical expected reach using inclusion-exclusion principle.
        For large n, this approximates to: E[R(n)] ≈ M * (1 - exp(-n/M))
        where M is the number of active users.
        """
        prob_not_reached = (1 - self.probabilities) ** n_impressions
        expected_reach = self.max_reach - np.sum(prob_not_reached)
        
        return expected_reach

In [ ]:
simulator = StochasticReachSimulator(timespends)

N_IMPRESSIONS = 30_000
N_RUNS = 2_000

reach_curves = simulator.simulate_multiple_runs(N_IMPRESSIONS, N_RUNS)

## 4. Analyze Results

In [ ]:
mean_reach = np.mean(reach_curves, axis=0)
std_reach = np.std(reach_curves, axis=0)
percentile_5 = np.percentile(reach_curves, 5, axis=0)
percentile_95 = np.percentile(reach_curves, 95, axis=0)

impressions = np.arange(N_IMPRESSIONS + 1)

plt.figure(figsize=(12, 8))

for i in range(min(10, N_RUNS)):
    plt.plot(impressions, reach_curves[i], alpha=0.1, color='gray')

plt.plot(impressions, mean_reach, 'b-', linewidth=2, label='Mean reach')
plt.fill_between(impressions, percentile_5, percentile_95, 
                 alpha=0.2, color='blue', label='90% confidence interval')
plt.fill_between(impressions, percentile_15, percentile_85, 
                 alpha=0.2, color='red', label='90% confidence interval')                 

plt.axhline(y=simulator.max_reach, color='r', linestyle='--', 
            label=f'Maximum possible reach: {simulator.max_reach:,}')

plt.xlabel('Number of Impressions')
plt.ylabel('Reach (Unique Users)')
plt.title('Stochastic Reach Curve Simulation')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Reach Efficiency Analysis

In [ ]:
reach_pct_total = mean_reach / simulator.total_users * 100
reach_pct_active = mean_reach / simulator.max_reach * 100

marginal_reach = np.diff(mean_reach)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

ax1.plot(impressions, reach_pct_total, 'b-', linewidth=2, 
         label='% of total population')
ax1.plot(impressions, reach_pct_active, 'g-', linewidth=2, 
         label='% of active users')
ax1.set_xlabel('Number of Impressions')
ax1.set_ylabel('Reach (%)')
ax1.set_title('Reach as Percentage of Population')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.semilogy(impressions[1:1000], marginal_reach[:999], 'r-', linewidth=2)
ax2.set_xlabel('Number of Impressions')
ax2.set_ylabel('Marginal Reach (log scale)')
ax2.set_title('Marginal Reach (Additional Users per Impression)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Theoretical vs Empirical Comparison

In [ ]:
impression_points = np.logspace(1, np.log10(N_IMPRESSIONS), 50).astype(int)
theoretical_reach = []

for n in impression_points:
    expected = simulator.compute_expected_reach(n)
    theoretical_reach.append(expected)

theoretical_reach = np.array(theoretical_reach)

plt.figure(figsize=(12, 8))

plt.plot(impressions, mean_reach, 'b-', linewidth=2, 
         label='Empirical (simulation mean)', alpha=0.8)

plt.plot(impression_points, theoretical_reach, 'ro--', 
         markersize=6, linewidth=2, label='Theoretical expected reach')

plt.axhline(y=simulator.max_reach, color='g', linestyle='--', 
            label=f'Maximum: {simulator.max_reach:,}')

plt.xlabel('Number of Impressions')
plt.ylabel('Reach (Unique Users)')
plt.title('Theoretical vs Empirical Reach Curves')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, N_IMPRESSIONS)
plt.show()

## 7. User Segment Analysis

In [ ]:
n_impressions_detail = 20000
reached_users_tracker = {}
impression_counts = np.zeros(simulator.total_users)

for i in range(n_impressions_detail):
    selected_user = np.random.choice(simulator.active_indices, p=simulator.probabilities)
    impression_counts[selected_user] += 1
    
    if selected_user not in reached_users_tracker:
        reached_users_tracker[selected_user] = i + 1

active_timespends = timespends[timespends > 0]
quartiles = np.percentile(active_timespends, [25, 50, 75])

user_categories = np.zeros(len(timespends))
user_categories[timespends == 0] = 0
user_categories[(timespends > 0) & (timespends <= quartiles[0])] = 1
user_categories[(timespends > quartiles[0]) & (timespends <= quartiles[1])] = 2  
user_categories[(timespends > quartiles[1]) & (timespends <= quartiles[2])] = 3
user_categories[timespends > quartiles[2]] = 4

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

categories = ['Zero', 'Low', 'Med-Low', 'Med-High', 'High']
avg_impressions = []
reach_rates = []

for cat in range(5):
    mask = user_categories == cat
    avg_impressions.append(np.mean(impression_counts[mask]))
    
    if cat == 0:
        reach_rates.append(0)
    else:
        users_in_cat = np.where(mask)[0]
        reached_in_cat = sum(1 for u in users_in_cat if u in reached_users_tracker)
        reach_rates.append(reached_in_cat / len(users_in_cat) * 100)

ax1.bar(categories, avg_impressions, color=['red', 'orange', 'yellow', 'lightgreen', 'green'])
ax1.set_xlabel('User Category (by timespend)')
ax1.set_ylabel('Average Impressions per User')
ax1.set_title(f'Impression Distribution ({n_impressions_detail:,} total impressions)')

ax2.bar(categories, reach_rates, color=['red', 'orange', 'yellow', 'lightgreen', 'green'])
ax2.set_xlabel('User Category (by timespend)')
ax2.set_ylabel('Reach Rate (%)')
ax2.set_title('Reach Rate by User Category')

plt.tight_layout()
plt.show()

## 8. Optimization Insights

In [ ]:
reach_targets = [0.3, 0.4, 0.5, 0.6, 0.7]
optimal_impressions = []

for target in reach_targets:
    target_reach = target * simulator.max_reach
    idx = np.where(mean_reach >= target_reach)[0]
    if len(idx) > 0:
        optimal_impressions.append(idx[0])
    else:
        optimal_impressions.append(None)

print("Reach Optimization Report")
print("=" * 50)
print(f"Total users: {simulator.total_users:,}")
print(f"Active users: {simulator.max_reach:,} ({simulator.max_reach/simulator.total_users*100:.1f}%)")
print("\nOptimal impression counts for reach targets:")
print("-" * 50)

for target, impressions in zip(reach_targets, optimal_impressions):
    if impressions is not None:
        actual_reach = mean_reach[impressions]
        efficiency = actual_reach / impressions if impressions > 0 else 0
        print(f"Target: {target*100:.0f}% of active users ({target*simulator.max_reach:.0f} users)")
        print(f"  → Impressions needed: {impressions:,}")
        print(f"  → Actual reach: {actual_reach:.0f} ({actual_reach/simulator.total_users*100:.1f}% of total)")
        print(f"  → Efficiency: {efficiency:.4f} unique users per impression")
        print()

## 9. Key Findings and Recommendations

1. **Non-linear growth**: Reach grows rapidly initially then saturates
2. **Diminishing returns**: Marginal reach decreases exponentially
3. **User heterogeneity**: High-timespend users receive disproportionate impressions
4. **Coverage limit**: Maximum reach is constrained by active user population

1. **Early impressions are most valuable**: Focus budget on initial reach
2. **Frequency capping**: Consider limiting impressions per user to improve efficiency
3. **Segment targeting**: Target low-timespend users explicitly to improve coverage
4. **Budget allocation**: Use marginal reach curves to optimize spend

- The reach process follows approximately: $R(n) ≈ M(1 - e^{-λn/M})$
- Where $M$ is max reach and $λ$ relates to the heterogeneity of user timespends
- High variance in timespends leads to slower saturation

## 10. Interesting questions
1. What is expected number of impressions to achieve all active users?
2. What is the probability of reach equal to zero for k impressions?
3. Are there other alternatives or models?